# Air-to-Air Heat Pump Design

In [1]:
from hvac import Quantity
from hvac.fluids import Fluid
from hvac.fluids import HumidAir

from design_parameters import CondenserParams, EvaporatorParams

Q_ = Quantity
R410A = Fluid("R410A")

## Heat Pump Design Parameters

Target heating capacity of the heat pump at rated operating conditions.

- state of indoor air entering the condenser
- state of outdoor air entering the evaporator

In [2]:
cnd_Q_dot = Q_(3.28, 'kW')
cnd_air_in = HumidAir(Tdb=Q_(20, 'degC'), RH=Q_(50, 'pct'))
evp_air_in = HumidAir(Tdb=Q_(7, 'degC'), Twb=Q_(6, 'degC'))

### Condenser Design Parameters

Determine the parameters to design the condenser.

- condensing temperature of the refrigerant at design operating conditions
- mass and volume flow rate of indoor air through the condenser (note: volume flow rate is referred to the air inlet)
- width and height of the condenser face area

In [3]:
cnd_params = CondenserParams(
    air_in=cnd_air_in,
    Q_dot=cnd_Q_dot,
    aspect_ratio=1/3
)

print(cnd_params)

air in: 20.00 °C DB, 7.29 g/kg AH (50 % RH)
air out: 30.00 °C DB, 7.29 g/kg AH (28 % RH)
TD in: 15.0 K
TD out: 5.0 K
T condensing: 35.0 °C
aspect ratio air face area: 1:3
---------------
heat transfer rate: 3.280 kW
air m-rate: 1157.631 kg/h
air v-rate: 972.248 m³/h
air face velocity: 2.0 m/s
air face width: 636 mm
air face height: 212 mm


### Evaporator Design Parameters

Determine the available parameters to design the evaporator.

- evaporating temperature of the refrigerant at design operating conditions
- degree of superheating of the refrigerant leaving the evaporator

The mass and volume flow rate of outdoor air through the evaporator cannot yet be determined as the required heat absorption rate of the refrigerant in the evaporator is still unknown.

In [4]:
evp_params = EvaporatorParams(
    air_in=evp_air_in,
    aspect_ratio=1/3
)

print(evp_params)

air in: 7.00 °C DB, 5.41 g/kg AH (87 % RH)
air out: 2.00 °C DB, 4.38 g/kg AH (100 % RH)
TD in: 10.0 K
TD out: 5.0 K
T evaporating: -3.0 °C
degree of superheating: 8.0 K
aspect ratio air face area: 1:3


With the chosen evaporating and the condensing temperature of the refrigerant, a compressor can be selected for the heat pump.

In [5]:
print(
    f"condensing temperature = {cnd_params.T_cnd.to('degC'):~P.0f}",
    f"evaporating temperature = {evp_params.T_evp.to('degC'):~P.0f}",
    sep="\n"
)

condensing temperature = 35 °C
evaporating temperature = -3 °C


## Compressor Selection

A refrigerant and compressor are selected using the compressor selection software program of a compressor manufacturer. To model the heat pump, the polynomial coefficients of the compressor performance curves (cooling capacity, mechanical power, and mass flow rate of refrigerant) are needed. These coefficients can then be entered into a CSV-file.

In [6]:
from hvac.vapor_compression.real_compressor import VariableSpeedCompressor

Define the selected compressor in this notebook:

In [7]:
compressor = VariableSpeedCompressor(
    coeff_file="VRJ028-K_R410A.csv",
    refrigerant=R410A,
    dT_sh=Q_(8, 'K'),
    dT_sc=Q_(2, 'K'),
    units={"n": "rps"}
)

Set the design operation parameters of the compressor (evaporating temperature, condensing temperature, and compressor speed):

In [8]:
compressor.T_evp = evp_params.T_evp
compressor.T_cnd = cnd_params.T_cnd
compressor.speed = Q_(1358, "rpm")

print(
    f"refrigerant mass flow rate: {compressor.m_dot.to('kg / hr'):~P.3f}",
    f"evaporator heat absorption rate: {compressor.Q_dot_evp.to('kW'):~P.3f}",
    f"condenser heat rejection rate: {compressor.Q_dot_cnd.to('kW'):~P.3f}",
    f"compressor absorbed power: {compressor.W_dot.to('kW'):~P.3f}",
    sep="\n"
)

refrigerant mass flow rate: 51.044 kg/h
evaporator heat absorption rate: 2.466 kW
condenser heat rejection rate: 3.255 kW
compressor absorbed power: 0.789 kW


> Evaporation temperature and condensing temperature are kept fixed. In case of a variable-speed compressor, compressor speed can be modified so that the condenser heat rejection rate would match with the target heating capacity of the heat pump.

We can check the refrigerant states in the standard vapor compression cycle:

In [9]:
import pandas as pd

with pd.option_context("display.max_columns", None, "display.width", 300):
    print(compressor.get_refrigerant_cycle())

                    T [degC]    P [bar]  rho [kg / m**3]  h [kJ / kg]  s [kJ / kg / K]
evaporator inlet   -3.075017   7.244576       100.464057   253.544550         1.198549
evaporator outlet   5.000000   7.244576        26.303711   428.961532         1.847484
condenser inlet    80.839913  21.383272        62.859263   484.601135         1.918419
condenser outlet   33.000000  21.447142      1018.120973   253.544550         1.180562


## Outdoor Air Mass and Volume Flow Rate

Once the compressor is selected, the absorption heat rate of the refrigerant in the evaporator is also known. Then, the mass and volume flow rate of outdoor air through the evaporator, and the width and height of the evaporator's face area can be determined.

In [10]:
evp_params.Q_dot = compressor.Q_dot_evp

print(evp_params)

air in: 7.00 °C DB, 5.41 g/kg AH (87 % RH)
air out: 2.00 °C DB, 4.38 g/kg AH (100 % RH)
TD in: 10.0 K
TD out: 5.0 K
T evaporating: -3.0 °C
degree of superheating: 8.0 K
aspect ratio air face area: 1:3
---------------
heat transfer rate: 2.466 kW
air m-rate: 1160.031 kg/h
air v-rate: 928.148 m³/h
air face velocity: 2.0 m/s
air face width: 622 mm
air face height: 207 mm


## Evaporator Sizing (Plain Fin Tube Counter Flow Air Evaporator)

In [11]:
from hvac.heat_exchanger.recuperator.fintube import PFT_CF_AE as Evaporator

Define the heat exchanger geometry of the evaporator. For now, the number of rows of the evaporator is just an initial guess.

In [12]:
evaporator = Evaporator(
    W_fro=evp_params.width,        # width of frontal area
    H_fro=evp_params.height,       # height of frontal area
    N_rows=3,                      # number of rows: initial guess
    S_trv=Q_(25.4, 'mm'),          # vertical distance between tubes
    S_lon=Q_(22.0, 'mm'),          # horizontal distance between tubes
    D_int=Q_(4.211, 'mm'),         # inner tube diameter
    D_ext=Q_(5.1, 'mm'),           # outer tube diameter
    t_fin=Q_(0.3302, 'mm'),        # fin thickness
    N_fin=1 / Q_(3.175, 'mm'),     # fin density
    k_fin=Q_(237, 'W / (m * K)'),  # conductivity of fin material
    num_circuits=1                 # number of parallel refrigerant circuits
)

The flow length and the number of rows of the evaporator should be determined so that refrigerant would leave the evaporator with the required degree of superheating at the specified operating conditions of the evaporator. These operating conditions are:

In [13]:
print(
    f"Entering air: {evp_params.air_in}",
    f"Mass flow rate of air: {evp_params.air_m_dot.to('kg / hr'):~P.3f}",
    f"Entering refrigerant (mixture): {compressor.mixture.T.to('degC'):~P.1f}, {compressor.mixture.x.to('pct'):~P.1f}",
    f"Mass flow rate of refrigerant: {compressor.m_dot.to('kg / hr'):~P.3f}",
    sep="\n"
)

Entering air: 7.00 °C DB, 5.41 g/kg AH (87 % RH)
Mass flow rate of air: 1160.031 kg/h
Entering refrigerant (mixture): -3.1 °C, 25.9 %
Mass flow rate of refrigerant: 51.044 kg/h


In [14]:
L_flow, N_rows = evaporator.solve(
    air_in=evp_params.air_in,
    air_m_dot=evp_params.air_m_dot,
    rfg_in=compressor.mixture,
    dT_sh=compressor.dT_sh,
    rfg_m_dot=compressor.m_dot
)

In [15]:
print(
    f"required flow length: {L_flow.to('mm'):~P.3f}",
    f"required number of rows: {N_rows}",
    sep="\n"
)

required flow length: 58.480 mm
required number of rows: 3


The mass flow rate of refrigerant in a single tube of the evaporator and also the Reynolds number of the refrigerant flow in the boiling part of the evaporator should be checked. A high Reynolds number (> 2300) is desirable to obtain an efficient heat transfer between air and refrigerant. By setting the number of refrigerant circuits when defining the heat exchanger geometry of the evaporator, we can rise/lower the mass flow rate of refrigerant in the tubes.

In [16]:
print(evaporator.boiling_region.core.internal.tube.m_dot.to('kg / hr'))
print(evaporator.boiling_region.core.internal.tube.Re)

51.04419877641644 kilogram / hour
9286.456039375433


After the evaporator has been sized, the flow length of the evaporator will be set according to the calculated required number of rows and the longitudinal pitch between rows.

In [17]:
print(evaporator.L_flow.to('mm'))
print(evaporator.N_rows)

66.0 millimeter
3.0


As the final flow length of the evaporator is rounded according to the number of rows, the mass flow rate of refrigerant that needs to flow through the evaporator to superheat the refrigerant to the required degree, will be changed. When the `solve()` method of the evaporator is called without specifying the mass flow rate of refrigerant, the mass flow rate of refrigerant is calculated so that refrigerant would leave the evaporator with the required degree of superheating.

In [18]:
rfg_m_dot = evaporator.solve(
    air_in=evp_params.air_in,
    air_m_dot=evp_params.air_m_dot,
    rfg_in=compressor.mixture,
    dT_sh=compressor.dT_sh
)

In [19]:
print(rfg_m_dot.to('kg / hr'))

56.651121936144435 kilogram / hour


## Condenser Sizing (Plain Fin Tube Counter Flow Air Condenser)

In [20]:
from hvac.heat_exchanger.recuperator.fintube import PFT_CF_AC as Condenser

The condenser is sized in the same way as the evaporator. To size the condenser, we need to set the degree of subcooling of refrigerant leaving the condenser.

In [21]:
condenser = Condenser(
    W_fro=cnd_params.width,
    H_fro=cnd_params.height,
    N_rows=4,
    S_trv=Q_(25.4, 'mm'),
    S_lon=Q_(22.0, 'mm'),
    D_int=Q_(4.211, 'mm'),
    D_ext=Q_(5.1, 'mm'),
    t_fin=Q_(0.3302, 'mm'),
    N_fin=1 / Q_(3.175, 'mm'),
    k_fin=Q_(237, 'W / (m * K)'),
    num_circuits=1
)

In [22]:
L_flow, N_rows = condenser.solve(
    air_in=cnd_params.air_in,
    air_m_dot=cnd_params.air_m_dot,
    rfg_in=compressor.discharge_gas,
    rfg_m_dot=compressor.m_dot,
    dT_sc=Q_(2, 'K')
)

In [23]:
print(
    f"required flow length of condenser: {L_flow.to('mm'):~P.0f}",
    f"number of rows: {N_rows}",
    sep="\n"
)

required flow length of condenser: 63 mm
number of rows: 3


In [24]:
print(condenser.subcooling_region.core.internal.tube.m_dot.to('kg / hr'))
print(condenser.subcooling_region.core.internal.tube.reynolds_number())

51.04419877641644 kilogram / hour
40542.80713632707


After sizing the condenser, the actual flow length of the condenser has been set according to the number of rows and the longitudinal pitch between rows:

In [25]:
print(condenser.L_flow.to('mm'))
print(condenser.N_rows)

66.0 millimeter
3.0


## Rated Heat Pump Performance

After the compressor has been selected and the evaporator and condenser have been sized, the heat pump as a whole still needs to be balanced so that the mass and energy balance of the vapor compression cycle will be satisfied. Mass balance implies that the mass flow rate of refrigerant let through by the expansion device –so that refrigerant leaves the evaporator with the required degree of superheat– must be equal to the mass flow rate of refrigerant displaced by the compressor. Energy balance implies that the sum of the heat rate absorbed by the refrigerant in the evaporator and the mechanical power supplied to the refrigerant by the compressor must be equal to the heat rate rejected by the refrigerant in the condenser.

In [26]:
from hvac.vapor_compression import SS_VCM as HeatPump

> The abbreviation `SS_VCM` stands for "Single Stage Vapor Compression Machine".

Heat pump model:

In [27]:
heat_pump = HeatPump(
    evaporator, condenser, compressor,
    n_cmp_min=Q_(900, 'rpm'),
    n_cmp_max=Q_(4200, 'rpm')
)

The balancing of the heat pump is called "rating". Rating a heat pump means that the steady-state performance characteristics of the heat pump are determined at the given operating conditions (state of inlet outdoor air at the evaporator, outdoor air mass flow rate, state of inlet indoor air at the condenser, indoor air mass flow rate, and compressor speed). To start the rating routine, it needs an initial guess of the evaporating and the condensing temperature of the refrigerant. To determine the mass and energy balance point of the heat pump, two different solvers are implemented. The default solver uses Scipy's `least_squares` function. The other solver uses Scipy's `minimize` function, which is more accurate, but runs much slower than `least_squares`.

In [28]:
output = heat_pump.rate(
    evp_air_in=evp_params.air_in,
    evp_air_m_dot=evp_params.air_m_dot,
    cnd_air_in=cnd_params.air_in,
    cnd_air_m_dot=cnd_params.air_m_dot,
    n_cmp=compressor.speed,
    T_evp_ini=compressor.T_evp,
    T_cnd_ini=compressor.T_cnd
)

[10032 | hvac.vapor_compression.machine | INFO] Starting machine operation analysis with...
[10032 | hvac.vapor_compression.machine | INFO] n_cmp = 23 rps
[10032 | hvac.vapor_compression.machine | INFO] evp_air_in: 7.00 °C, 87 %
[10032 | hvac.vapor_compression.machine | INFO] evp_air_m_dot: 1160.03 kg/h
[10032 | hvac.vapor_compression.machine | INFO] cnd_air_in: 20.00 °C, 50 %
[10032 | hvac.vapor_compression.machine | INFO] cnd_air_m_dot: 1157.63 kg/h
[10032 | hvac.vapor_compression.machine | INFO] Iteration 1: Try with: T_evp = -3.000 °C, T_cnd = 35.000 °C
[10032 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant mass flow rate from compressor = 51.044 kg/h
[10032 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant entering condenser with T = 80.840 °C, P = 21.383 bar
[10032 | hvac.vapor_compression.machine | INFO] Iteration 1: Refrigerant leaving condenser with T = 30.504 °C, P = 21.383 bar
[10032 | hvac.vapor_compression.machine | INFO] Iteration 1: Refr

When the rating routine has finished, the steady-state heat pump performance at rated design conditions is now fully determined.

In [29]:
print(output.to_text())

evp_air_m_dot = 1160.031 kg/h
cnd_air_m_dot = 1157.631 kg/h
evp_air_in = 7.000 °C DB, 5.410 g/kg AH (87 % RH)
cnd_air_in = 20.000 °C DB, 7.294 g/kg AH (50 % RH)
n_cmp = 1358.000 rpm
dT_sh = 8.000 K
evp_air_out = 1.738 °C DB, 4.300 g/kg AH (100 % RH)
cnd_air_out = 30.438 °C DB, 7.294 g/kg AH (27 % RH)
evp_Q_dot = 2.618 kW
cnd_Q_dot = 3.424 kW
cmp_W_dot = 0.803 kW
rfg_m_dot = 51.619 kg/h
COP = 4.262 frac
EER = 3.259 frac
evp_eps = 0.5099 frac
cnd_eps = 0.5374 frac
T_evp = -2.655 °C
P_evp = 7.327 bar
T_cnd = 35.560 °C
P_cnd = 21.749 bar
dT_sc = 6.612 K
suction_gas = 5.345 °C, 429.093 kJ/kg, 7.327 bar
discharge_gas = 81.726 °C, 485.118 kJ/kg, 21.749 bar
liquid = 28.948 °C, 246.348 kJ/kg, 21.749 bar
mixture = -2.733 °C, 246.348 kJ/kg, 7.327 bar, 22 %
evp_air_dP = 14.814 Pa
cnd_air_dP = 15.654 Pa

